In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - plot_density_posterior.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make a figure of the density posteriors for the paper
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, time, tqdm, warnings, multiprocessing, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt
import corner

# mwdust, isodist stuff, galpy stuff
import mwdust
from isodist import FEH2Z, Z2FEH
from galpy.util import coords
from galpy import orbit

# Fitting, optimization and statistics
import emcee
import scipy.optimize

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import iso as piso
from ges_mass import util as putil
from ges_mass import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max], ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.8]
# dfs = putil.load_distribution_functions(df_filename, betas)

In [ ]:
fig_dir = './fig/'

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

### More parameters

In [ ]:
# Arrays to hold information about the different HaloFit objects used for
# plotting
n_profile = 4
theta_in_degr = False
phi_in_degr = True
rad_to_degr = 180./np.pi
selec = [None,'eLz','AD','JRLz']
fit_type = ['all','gse','gse','gse']
sample_labels = ['Halo Sample','$e-L_\mathrm{z}$',r'AD',r'$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$']
# These are the density profiles that will be shown for the above. They should
# be the best-fits.
densfuncs = [pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             ]
# Shortcut if all versions are the same
_version = '100w_1e4n'
versions = [_version]*n_profile
# versions = ['100w_2e3n','100w_2e3n','100w_2e3n','1002_2e3n']
colors = ['Black','Red','DarkOrange','DodgerBlue']

### Mapping from density profile to parameter index
Do this so that all sets of parameters can be cast in the same array shape. Each map should be a dictionary with keys as the density function and values as a list of indices of the parameters: alpha1, alpha2, r1, p, q, eta, theta, phi, fdisk. Use -1 if the parameter doesn't exist for that density function. Should make another mapping if showing double broken power laws


In [ ]:
## Mapping and other variables for up to exponential breaks (includes r1)
n_param = 8
# Maps alpha1,r1,p,q,theta,eta,phi,fdisk
densfunc_map = {'triaxial_single_angle_zvecpa':[0,-1,1,2,3,4,5,-1],
                'triaxial_single_angle_zvecpa_plusexpdisk':[0,-1,1,2,3,4,5,6],
                'triaxial_single_cutoff_zvecpa':[0,1,2,3,4,5,6,-1],
                'triaxial_single_cutoff_zvecpa_plusexpdisk':[0,1,2,3,4,5,6,7],}
for key in densfunc_map.keys():
    assert len(densfunc_map[key]) == n_param

domain = [[-5,4],
          [0,20],
          [0,1],
          [0,1],
          [0,2*np.pi],  
          [0.5,1],
          [0,np.pi],
          [0,1]]
labels = [r'$\alpha_{1}$', r'$r_{1}$ [kpc]', r'$p$', 
          r'$q$', r'$\theta$', r'$\eta$', r'$\phi$','$f_\mathrm{disk}$']
if theta_in_degr:
    domain[5] = [domain[4][0]*rad_to_degr,domain[4][1]*rad_to_degr]
    labels[5] += ' [deg]'
else:
    labels[5] += ' [rad]'
if phi_in_degr:
    domain[7] = [domain[6][0]*rad_to_degr,domain[6][1]*rad_to_degr]
    labels[7] += ' [deg]'
else:
    labels[7] += ' [rad]'
    
sigmas = [1,2]
levels = [1 - np.exp(-(s)**2/2) for s in sigmas]

In [ ]:
# Plotting and HaloFit kwargs
n_bins = 10
nrand = 100
plot_physical = True
plot_lines = False
plot_68perc = True
verbose = False
hf_kwargs['verbose'] = verbose

# Fontsize and formatting kwargs
ylabel_fs = 12
xlabel_fs = 12
posterior_lw = 1
legend_fs = 8
ticklabel_fs = 10
annotate_fs = 10
plot_lw = 0.5
plot_alpha = 0.2
perc_alpha = 1.
perc_lw = 1.5
perc_alpha_fill = 0.2
columnwidth = 244./72.27 # Only one column. In inches, from pt
textwidth = 508./72.27 # Whole page. In inches, from pt

# Effective radius grid
re_range = [2.,55.]
nre = 100
re = np.logspace(np.log10(re_range[0]), np.log10(re_range[1]), nre)
_phi = np.zeros_like(re)
_z = np.zeros_like(re)
log_re = np.log10(re)

# Make the figure
fig = plt.figure(figsize=(columnwidth,3))
ax = fig.add_subplot(111)

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    # Calculate density and plot
    if plot_lines:
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if plot_physical:
                dens *= hf.facs[randind[j]]
            ax.plot(re, np.log10(dens), linewidth=plot_lw,
                    color=colors[i], alpha=plot_alpha
                    )
        ax.plot([],[],linewidth=plot_lw,color=colors[i],alpha=plot_alpha,
                label=sample_labels)
    elif plot_68perc:
        dens_dist = np.zeros((nrand,nre))
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if 'plusexpdisk' in hf.densfunc.__name__:
                dens *= (1-_sample[-1])
            if plot_physical:
                dens *= hf.facs[randind[j]]
            dens_dist[j] = dens
        dens_lb,dens_med,dens_ub = np.quantile(dens_dist,[0.16,0.5,0.84],
                                                 axis=0)
        ax.plot(re, np.log10(dens_med), linewidth=perc_lw,
                color=colors[i], alpha=perc_alpha, label=sample_labels[i])
        ax.fill_between(re, np.log10(dens_lb), np.log10(dens_ub), 
                        color=colors[i], alpha=perc_alpha_fill)
        if i == 0:
            ax.plot(re, np.log10(dens_med/2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
            ax.plot(re, np.log10(dens_med*2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
    
ax.set_xlabel(r'$m$ [kpc]',fontsize=xlabel_fs)
if plot_physical:
    ax.set_ylabel(r'$\log_{10} (\rho / \mathrm{M}_{\odot} \mathrm{kpc}^{-3})$',
                    fontsize=ylabel_fs)
else:
    ax.set_ylabel(r'$\log_{10} (\nu_{\star})$', fontsize=ylabel_fs)
ax.set_xscale('log')
ax.set_xlim(re_range[0],re_range[1])
ax.set_ylim(0.,7.)
ax.legend(loc='best', fontsize=legend_fs, frameon=True)
ax.axvline(ro, color='k', linestyle='dotted', linewidth=1., alpha=0.5)

fig.tight_layout()
fig.subplots_adjust(bottom=0.175,left=0.15)
fig.savefig(fig_dir+'pdensity.pdf')
fig.show()

In [ ]:
# Plotting and HaloFit kwargs
n_bins = 10
nrand = 100
plot_physical = True
plot_lines = False
plot_68perc = True
verbose = False
hf_kwargs['verbose'] = verbose

# Fontsize and formatting kwargs
ylabel_fs = 12
xlabel_fs = 12
posterior_lw = 1
legend_fs = 8
ticklabel_fs = 10
annotate_fs = 10
plot_lw = 0.5
plot_alpha = 0.2
perc_alpha = 1.
perc_lw = 1.5
perc_alpha_fill = 0.2
columnwidth = 244./72.27 # Only one column. In inches, from pt
textwidth = 508./72.27 # Whole page. In inches, from pt

# Effective radius grid
re_range = [2.,55.]
nre = 100
re = np.logspace(np.log10(re_range[0]), np.log10(re_range[1]), nre)
_phi = np.zeros_like(re)
_z = np.zeros_like(re)
log_re = np.log10(re)

# Make the figure
fig = plt.figure(figsize=(columnwidth,6))
ax1,ax2 = fig.subplots(nrows=2,ncols=1)

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    # Calculate density and plot
    if plot_lines:
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so don't set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if plot_physical:
                dens *= hf.facs[randind[j]]
            ax1.plot(re, np.log10(dens), linewidth=plot_lw,
                    color=colors[i], alpha=plot_alpha
                    )
        ax1.plot([],[],linewidth=plot_lw,color=colors[i],alpha=plot_alpha,
                label=sample_labels)
    elif plot_68perc:
        dens_dist = np.zeros((nrand,nre))
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so don't set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if 'plusexpdisk' in hf.densfunc.__name__:
                dens *= (1-_sample[-1])
            if plot_physical:
                dens *= hf.facs[randind[j]]
            dens_dist[j] = dens
        dens_lb,dens_med,dens_ub = np.quantile(dens_dist,[0.16,0.5,0.84],
                                                 axis=0)
        ax1.plot(re, np.log10(dens_med), linewidth=perc_lw,
                color=colors[i], alpha=perc_alpha, label=sample_labels[i])
        ax1.fill_between(re, np.log10(dens_lb), np.log10(dens_ub), 
                        color=colors[i], alpha=perc_alpha_fill)
        if i == 0:
            ax1.plot(re, np.log10(dens_med/2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
            ax1.plot(re, np.log10(dens_med*2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
    
ax1.set_xlabel(r'$m$ [kpc]',fontsize=xlabel_fs)
if plot_physical:
    ax1.set_ylabel(r'$\log_{10} (\rho / \mathrm{M}_{\odot} \mathrm{kpc}^{-3})$',
                    fontsize=ylabel_fs)
else:
    ax1.set_ylabel(r'$\log_{10} (\nu_{\star})$', fontsize=ylabel_fs)
ax1.set_xscale('log')
ax1.set_xlim(re_range[0],re_range[1])
ax1.set_ylim(0.,7.)
ax1.legend(loc='best', fontsize=legend_fs, frameon=True)
ax1.axvline(ro, color='k', linestyle='dotted', linewidth=1., alpha=0.5)

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    # Calculate density and plot
    if plot_lines:
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing X, so don't set p and q to zero
            # _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if plot_physical:
                dens *= hf.facs[randind[j]]
            ax2.plot(re, np.log10(dens), linewidth=plot_lw,
                    color=colors[i], alpha=plot_alpha
                    )
        ax2.plot([],[],linewidth=plot_lw,color=colors[i],alpha=plot_alpha,
                label=sample_labels)
    elif plot_68perc:
        dens_dist = np.zeros((nrand,nre))
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing X, so don't set p and q to zero
            # _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if 'plusexpdisk' in hf.densfunc.__name__:
                dens *= (1-_sample[-1])
            if plot_physical:
                dens *= hf.facs[randind[j]]
            dens_dist[j] = dens
        dens_lb,dens_med,dens_ub = np.quantile(dens_dist,[0.16,0.5,0.84],
                                                 axis=0)
        ax2.plot(re, np.log10(dens_med), linewidth=perc_lw,
                color=colors[i], alpha=perc_alpha, label=sample_labels[i])
        ax2.fill_between(re, np.log10(dens_lb), np.log10(dens_ub), 
                        color=colors[i], alpha=perc_alpha_fill)
        if i == 0:
            ax2.plot(re, np.log10(dens_med/2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
            ax2.plot(re, np.log10(dens_med*2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
    
ax2.set_xlabel(r'$X_\mathrm{GC}$ [kpc]',fontsize=xlabel_fs)
if plot_physical:
    ax2.set_ylabel(r'$\log_{10} (\rho / \mathrm{M}_{\odot} \mathrm{kpc}^{-3})$',
                    fontsize=ylabel_fs)
else:
    ax2.set_ylabel(r'$\log_{10} (\nu_{\star})$', fontsize=ylabel_fs)
ax2.set_xscale('log')
ax2.set_xlim(re_range[0],re_range[1])
ax2.set_ylim(0.,7.)
# ax2.legend(loc='best', fontsize=legend_fs, frameon=True)
ax2.axvline(ro, color='k', linestyle='dotted', linewidth=1., alpha=0.5)

fig.tight_layout()
fig.subplots_adjust(bottom=0.1,left=0.15)
fig.savefig(fig_dir+'pdensity_gcx.pdf')
fig.show()

### Also show BPL models for eLz and AD

In [ ]:
# Arrays to hold information about the different HaloFit objects used for
# plotting
n_profile = 6
theta_in_degr = False
phi_in_degr = True
rad_to_degr = 180./np.pi
selec = [None,'eLz','AD','JRLz','eLz','AD']
fit_type = ['all','gse','gse','gse','gse','gse']
sample_labels = ['Halo Sample','$e-L_\mathrm{z}$',r'AD',r'$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$','$e-L_\mathrm{z}$',r'AD']
# These are the density profiles that will be shown for the above. They should
# be the best-fits.
densfuncs = [pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa
             ]
# Shortcut if all versions are the same
_version = '100w_1e4n'
versions = [_version]*n_profile
# versions = ['100w_2e3n','100w_2e3n','100w_2e3n','1002_2e3n']
colors = ['Black','Red','DarkOrange','DodgerBlue','Crimson','Orange']
linestyles = ['solid','solid','solid','solid','dashed','dashed']
plot_fill_for_68perc = [True,True,True,True,False,False]

In [ ]:
# Plotting and HaloFit kwargs
n_bins = 10
nrand = 100
plot_physical = True
plot_lines = False
plot_68perc = True
verbose = False
hf_kwargs['verbose'] = verbose

# Fontsize and formatting kwargs
ylabel_fs = 12
xlabel_fs = 12
posterior_lw = 1
legend_fs = 8
ticklabel_fs = 10
annotate_fs = 10
plot_lw = 0.5
plot_alpha = 0.2
perc_alpha = 1.
perc_lw = 1.5
perc_alpha_fill = 0.2
columnwidth = 244./72.27 # Only one column. In inches, from pt
textwidth = 508./72.27 # Whole page. In inches, from pt

# Effective radius grid
re_range = [2.,55.]
nre = 100
re = np.logspace(np.log10(re_range[0]), np.log10(re_range[1]), nre)
_phi = np.zeros_like(re)
_z = np.zeros_like(re)
log_re = np.log10(re)

# Make the figure
fig = plt.figure(figsize=(columnwidth,6))
ax1,ax2 = fig.subplots(nrows=2,ncols=1)

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    # Calculate density and plot
    if plot_lines:
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so don't set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if plot_physical:
                dens *= hf.facs[randind[j]]
            ax1.plot(re, np.log10(dens), linewidth=plot_lw,
                    color=colors[i], alpha=plot_alpha,
                    linestyle=linestyles[i])
        ax1.plot([],[],linewidth=plot_lw,color=colors[i],alpha=plot_alpha,
                label=sample_labels)
    elif plot_68perc:
        dens_dist = np.zeros((nrand,nre))
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing re, so don't set p and q to zero
            _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if 'plusexpdisk' in hf.densfunc.__name__:
                dens *= (1-_sample[-1])
            if plot_physical:
                dens *= hf.facs[randind[j]]
            dens_dist[j] = dens
        dens_lb,dens_med,dens_ub = np.quantile(dens_dist,[0.16,0.5,0.84],
                                                 axis=0)
        ax1.plot(re, np.log10(dens_med), linewidth=perc_lw,
                color=colors[i], alpha=perc_alpha, label=sample_labels[i],
                linestyle=linestyles[i])
        if plot_fill_for_68perc[i]:
            ax1.fill_between(re, np.log10(dens_lb), np.log10(dens_ub), 
                            color=colors[i], alpha=perc_alpha_fill)
        if i == 0:
            ax1.plot(re, np.log10(dens_med/2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
            ax1.plot(re, np.log10(dens_med*2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
    
ax1.set_xlabel(r'$m$ [kpc]',fontsize=xlabel_fs)
if plot_physical:
    ax1.set_ylabel(r'$\log_{10} (\rho / \mathrm{M}_{\odot} \mathrm{kpc}^{-3})$',
                    fontsize=ylabel_fs)
else:
    ax1.set_ylabel(r'$\log_{10} (\nu_{\star})$', fontsize=ylabel_fs)
ax1.set_xscale('log')
ax1.set_xlim(re_range[0],re_range[1])
ax1.set_ylim(0.,7.)
ax1.legend(loc='best', fontsize=legend_fs, frameon=True)
ax1.axvline(ro, color='k', linestyle='dotted', linewidth=1., alpha=0.5)

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    # Calculate density and plot
    if plot_lines:
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing X, so don't set p and q to zero
            # _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if plot_physical:
                dens *= hf.facs[randind[j]]
            ax2.plot(re, np.log10(dens), linewidth=plot_lw,
                    color=colors[i], alpha=plot_alpha
                    )
        ax2.plot([],[],linewidth=plot_lw,color=colors[i],alpha=plot_alpha,
                label=sample_labels)
    elif plot_68perc:
        dens_dist = np.zeros((nrand,nre))
        for j in range(nrand):
            _sample = samples_in[j]
            # Doing X, so don't set p and q to zero
            # _sample[pq_indx] = [1.,1.]
            dens = densfunc_nodisk(re,_phi,_z,params=_sample)
            if 'plusexpdisk' in hf.densfunc.__name__:
                dens *= (1-_sample[-1])
            if plot_physical:
                dens *= hf.facs[randind[j]]
            dens_dist[j] = dens
        dens_lb,dens_med,dens_ub = np.quantile(dens_dist,[0.16,0.5,0.84],
                                                 axis=0)
        ax2.plot(re, np.log10(dens_med), linewidth=perc_lw,
                color=colors[i], alpha=perc_alpha, label=sample_labels[i],
                linestyle=linestyles[i])
        if plot_fill_for_68perc[i]:
            ax2.fill_between(re, np.log10(dens_lb), np.log10(dens_ub), 
                            color=colors[i], alpha=perc_alpha_fill)
        if i == 0:
            ax2.plot(re, np.log10(dens_med/2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
            ax2.plot(re, np.log10(dens_med*2.), linewidth=perc_lw,
                color=colors[i], alpha=0.5, linestyle='dashed')
    
ax2.set_xlabel(r'$X_\mathrm{GC}$ [kpc]',fontsize=xlabel_fs)
if plot_physical:
    ax2.set_ylabel(r'$\log_{10} (\rho / \mathrm{M}_{\odot} \mathrm{kpc}^{-3})$',
                    fontsize=ylabel_fs)
else:
    ax2.set_ylabel(r'$\log_{10} (\nu_{\star})$', fontsize=ylabel_fs)
ax2.set_xscale('log')
ax2.set_xlim(re_range[0],re_range[1])
ax2.set_ylim(0.,7.)
# ax2.legend(loc='best', fontsize=legend_fs, frameon=True)
ax2.axvline(ro, color='k', linestyle='dotted', linewidth=1., alpha=0.5)

fig.tight_layout()
fig.subplots_adjust(bottom=0.1,left=0.15)
# fig.savefig(fig_dir+'pdensity_gcx.pdf')
fig.show()

### Determine values of $\rho_{0}$

In [ ]:
hf.densfunc

In [ ]:
hf.samples[:,-2].max()

In [ ]:
selec = [None,'eLz','AD','JRLz']
fit_type = ['all','gse','gse','gse']
sample_labels = ['Halo Sample','$e-L_\mathrm{z}$','AD','$\sqrt{J_\mathrm{R}}-L_\mathrm{z}$']
# These are the density profiles that will be shown for the above. They should
# be the best-fits.
densfuncs = [pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk
             ]

# Keywords
n_profile = len(densfuncs)
nrand = 100

# Loop over each density profile
for i in range(n_profile):
    
    # Account for changing Fe/H range
    if fit_type[i] == 'gse':
        feh_range_fit = copy.deepcopy(feh_range_gse)
    else:
        feh_range_fit = copy.deepcopy(feh_range_all)
    hf = pmass.HaloFit(densfunc=densfuncs[i], fit_type=fit_type[i], 
                       version=versions[i], selec=selec[i], 
                       feh_range=feh_range_fit, **hf_kwargs)

    # Load the results
    hf.get_results()
    hf_samples = hf.samples
    hf_samples = pdens.denormalize_parameters(hf_samples,hf.densfunc,
        theta_in_degr=theta_in_degr, phi_in_degr=phi_in_degr)
    n_samples = hf_samples.shape[0]
    
    # Get the indices of the p and q parameters
    pq_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    
    if plot_physical:
        assert len(hf.facs) == len(hf.mass_inds)
        n_mass = len(hf.facs)
        if nrand is not None and nrand > len(hf.mass_inds):
            print('Setting nrand to the number of masses in hf')
            nrand = n_mass
            
    # Determine samples for which to calculate density
    if plot_physical:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,n_mass,size=nrand)
            samples_in = hf.samples[hf.mass_inds[randind]]
        else:
            samples_in = hf.samples[hf.mass_inds[:]]
    else:
        if nrand is not None:
            nrand = int(nrand)
            rnp = np.random.default_rng()
            randind = rnp.integers(0,hf.samples.shape[0],size=nrand)
            samples_in = hf.samples[randind]
        else:
            samples_in = hf.samples[:]
    
    # Get the density profile without a disk component if it has one
    densfunc_nodisk = pdens.get_densfunc_nodisk(hf.densfunc)
    
    dens_dist = np.zeros((nrand,nre))
    norms = np.zeros((nrand))
    for j in range(nrand):
        _sample = samples_in[j]
        dens = densfunc_nodisk(re,_phi,_z,params=_sample)
        norm = 1.
        if 'plusexpdisk' in hf.densfunc.__name__:
            norm *= (1-_sample[-1])
        if plot_physical:
            norm *= hf.facs[randind[j]]
        dens *= norm
        dens_dist[j] = dens
        norms[j] = norm
    
    print(sample_labels[i]+' normalizations in msun / cubic kpc')
    print(np.quantile(norms,[0.16,0.5,0.84]))
    print('')